In [1]:
from lightgbm import Dataset
from lightgbm import train
from pandas import DataFrame
from pandas import read_csv
from sklearn.model_selection import train_test_split

In [2]:
train_df = read_csv('../input/spaceship-titanic/train.csv')
print(train_df.columns.tolist())

# get the labels
y = train_df['Transported'].values
train_df = train_df.drop(columns=['PassengerId', 'Transported'], inplace=False)
train_df = train_df.drop(columns=['CryoSleep', 'Destination', 'HomePlanet', 'VIP', 'Cabin', 'Name',], inplace=False)

x = train_df.values

# split
random_state = 1
x, x_test, y, y_test = train_test_split(x, y, test_size=0.2, random_state=random_state, stratify=y)
train_data = Dataset(x, label=y, categorical_feature='auto')
test_data = Dataset(x_test, label=y_test)

['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported']


In [3]:
parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': -1
}

model = train(parameters, train_data, valid_sets=test_data, num_boost_round=10000,)

[1]	valid_0's auc: 0.732844
[2]	valid_0's auc: 0.792261
[3]	valid_0's auc: 0.795906
[4]	valid_0's auc: 0.798242
[5]	valid_0's auc: 0.800042
[6]	valid_0's auc: 0.806015
[7]	valid_0's auc: 0.809515
[8]	valid_0's auc: 0.81033
[9]	valid_0's auc: 0.814637
[10]	valid_0's auc: 0.818065
[11]	valid_0's auc: 0.817928
[12]	valid_0's auc: 0.820387
[13]	valid_0's auc: 0.821793
[14]	valid_0's auc: 0.822465
[15]	valid_0's auc: 0.824238
[16]	valid_0's auc: 0.824891
[17]	valid_0's auc: 0.826473
[18]	valid_0's auc: 0.826354
[19]	valid_0's auc: 0.827339
[20]	valid_0's auc: 0.827983
[21]	valid_0's auc: 0.82904
[22]	valid_0's auc: 0.829696
[23]	valid_0's auc: 0.828559
[24]	valid_0's auc: 0.829138
[25]	valid_0's auc: 0.829106
[26]	valid_0's auc: 0.829272
[27]	valid_0's auc: 0.831055
[28]	valid_0's auc: 0.831389
[29]	valid_0's auc: 0.832228
[30]	valid_0's auc: 0.8321
[31]	valid_0's auc: 0.831689
[32]	valid_0's auc: 0.831897
[33]	valid_0's auc: 0.831719
[34]	valid_0's auc: 0.831823
[35]	valid_0's auc: 0.83235

In [4]:
submission_df = read_csv('../input/spaceship-titanic/test.csv')
ids = submission_df['PassengerId'].values
columns = ['PassengerId', 'CryoSleep', 'Destination', 'HomePlanet', 'VIP', 'Cabin', 'Name',]
submission_df = submission_df.drop(columns=columns, inplace=False, axis=1)
output_df = DataFrame({'PassengerId': ids, 'Transported': model.predict(submission_df.values) > 0.5})
output_df.to_csv('submission.csv', index=False)